In [ ]:
# example of a dcgan on cifar10
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy import vstack
from numpy.random import randn
from numpy.random import randint
from keras.datasets.cifar10 import load_data
from tensorflow.keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import MaxPooling2D
from keras.layers import LeakyReLU
from keras.layers import Dropout
from matplotlib import pyplot
from keras.models import load_model
import numpy as npu

In [ ]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import cv2
import random
from tensorflow import keras
from keras.layers import Dense, Input, InputLayer, Flatten
from keras.models import Sequential, Model
from  matplotlib import pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver() # TPU detection
except ValueError: # If TPU not found
  tpu = None

# Select appropriate distribution strategy
if tpu:
  tf.config.experimental_connect_to_cluster(tpu)
  tf.tpu.experimental.initialize_tpu_system(tpu)
  tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])  
else:
  tpu_strategy = tf.distribute.get_strategy() # Default strategy that works on CPU and single GPU
  print('Running on CPU instead')
print("Number of accelerators: ", tpu_strategy.num_replicas_in_sync)  

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.95.130.122:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.95.130.122:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Running on TPU  ['10.95.130.122:8470']
Number of accelerators:  8


# pre-trained Isolated model (80,80,3) images

weights of model are from InceptionV3 pre-trained model


In [ ]:
model_path = "/content/drive/MyDrive/"
with tpu_strategy.scope():
    isolated_model = tf.keras.models.load_model(model_path+"InceptionV3_isolated_model_stanford_dogs_80x80x3.h5")

In [ ]:
IMG_HEIGHT=80
IMG_WIDTH=80

# **Load The Stanford Dogs Dataset**
The Stanford Dogs Dataset contains images of 120 breeds of dogs from around the world. This dataset has been built using images and annotation from ImageNet for the task of fine-grained image categorization.
Contents of the dataset:

Number of categories: 120
Number of images: 20,580
Annotations: Class labels, Bounding boxes

The dataset can be downloaded from http://vision.stanford.edu/aditya86/ImageNetDogs/.

In [ ]:
def create_dataset(dataset_folder):  
    img_data_array=[]
    class_name=[]
   
    for dir1 in os.listdir(dataset_folder):
        print(dir1)
        for file in os.listdir(os.path.join(dataset_folder, dir1)):
       
            image_path= os.path.join(dataset_folder, dir1,  file)
            image= cv2.imread( image_path, cv2.COLOR_BGR2RGB)
            image=cv2.resize(image, (IMG_HEIGHT, IMG_WIDTH),interpolation = cv2.INTER_AREA)
            image=np.array(image)
            image = image.astype('float32')
            image = (image-127.5) / 127.5 
            img_data_array.append(image)
            class_name.append(dir1)
    return img_data_array, class_name

In [ ]:
#load data
with tpu_strategy.scope():
  dataset_folder=r'/content/drive/MyDrive/Stanford_Dogs_dataset/Images/'
  x_train, y_train = create_dataset(dataset_folder)

n02088238-basset
n02085782-Japanese_spaniel
n02086910-papillon
n02085620-Chihuahua
n02088094-Afghan_hound
n02087046-toy_terrier
n02086240-Shih-Tzu
n02087394-Rhodesian_ridgeback
n02086646-Blenheim_spaniel
n02086079-Pekinese
n02085936-Maltese_dog
n02088364-beagle
n02088632-bluetick
n02089078-black-and-tan_coonhound
n02089867-Walker_hound
n02090379-redbone
n02090622-borzoi
n02091032-Italian_greyhound
n02091134-whippet
n02091467-Norwegian_elkhound
n02091831-Saluki
n02092339-Weimaraner
n02093256-Staffordshire_bullterrier
n02093428-American_Staffordshire_terrier
n02093647-Bedlington_terrier
n02093754-Border_terrier
n02093991-Irish_terrier
n02094258-Norwich_terrier
n02094433-Yorkshire_terrier
n02095314-wire-haired_fox_terrier
n02095570-Lakeland_terrier
n02095889-Sealyham_terrier
n02096051-Airedale
n02096177-cairn
n02096437-Dandie_Dinmont
n02096585-Boston_bull
n02097047-miniature_schnauzer
n02097130-giant_schnauzer
n02097298-Scotch_terrier
n02097474-Tibetan_terrier
n02097658-silky_terrier
n020

In [ ]:
x_train = np.asarray(x_train)
print(x_train.shape)

(20580, 80, 80, 3)


In [ ]:
dataset=x_train

# define global parameters

In [ ]:
im_model = tf.keras.Model(inputs=isolated_model.input,
                                       outputs=isolated_model.get_layer(index=-2).output)
o=im_model(dataset)
output_dim = o.shape[1]

TARGET_INDEX1 = 0
TARGET_INDEX2 = 1
mask = np.zeros(output_dim)
mask[TARGET_INDEX1] = 1
mask[TARGET_INDEX2] = 1

In [ ]:
o.shape

TensorShape([20580, 2048])

In [ ]:
_max=tf.reduce_max(o, axis=0)

# Discriminator defintion

In [ ]:
# define the standalone discriminator model
def define_discriminator(in_shape=(IMG_HEIGHT,IMG_WIDTH,3)):
	model = Sequential()
	# normal
	model.add(Conv2D(64, (3,3), padding='same', input_shape=in_shape))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Dropout(0.2))
	# downsample
	model.add(Conv2D(64, (3,3), strides=2, padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Dropout(0.2))
	# downsample
	model.add(Conv2D(128, (3,3), strides=2, padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Dropout(0.2))
	# downsample
	model.add(Conv2D(128, (3,3), strides=2, padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Dropout(0.2))
	model.add(Conv2D(256, (3,3), strides=2, padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Dropout(0.2))
	# classifier
	model.add(Flatten())
	model.add(Dense(1, activation='sigmoid'))
	# compile model
	opt = Adam(learning_rate=0.0001, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
	return model

In [ ]:
d= define_discriminator()
d.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 80, 80, 64)        1792      
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 80, 80, 64)        0         
                                                                 
 dropout (Dropout)           (None, 80, 80, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 40, 40, 64)        36928     
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 40, 40, 64)        0         
                                                                 
 dropout_1 (Dropout)         (None, 40, 40, 64)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 20, 20, 128)       7

# generator defintion

In [ ]:
# define the standalone generator model
def define_generator(latent_dim):
	model = Sequential()
	# foundation for 3x3 image
	n_nodes = 256 * 5 * 5
	model.add(Dense(n_nodes, input_dim=latent_dim+2))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Reshape((5, 5, 256)))
	model.add(Conv2DTranspose(128, (4,4), strides=2, padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Conv2DTranspose(128, (4,4), strides=2, padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Conv2DTranspose(128, (4,4), strides=2, padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Conv2DTranspose(128, (4,4), strides=2, padding='same'))
	model.add(LeakyReLU(alpha=0.2))

	model.add(Conv2D(3, (3,3), activation='tanh', padding='same'))
	return model

In [ ]:
g=define_generator(10)
g.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 6400)              83200     
                                                                 
 leaky_re_lu_5 (LeakyReLU)   (None, 6400)              0         
                                                                 
 reshape (Reshape)           (None, 5, 5, 256)         0         
                                                                 
 conv2d_transpose (Conv2DTra  (None, 10, 10, 128)      524416    
 nspose)                                                         
                                                                 
 leaky_re_lu_6 (LeakyReLU)   (None, 10, 10, 128)       0         
                                                                 
 conv2d_transpose_1 (Conv2DT  (None, 20, 20, 128)      262272    
 ranspose)                                            

# GAN defintion

In [ ]:
# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model):
	# make weights in the discriminator not trainable
	d_model.trainable = False
	# connect them
	model = Sequential()
	# add generator
	model.add(g_model)
	# add the discriminator
	model.add(d_model)
	# compile model
	opt = Adam(learning_rate=0.0001, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt)
	return model

# some functions for generating real and fake samples

In [ ]:
# select real samples
def generate_real_samples(dataset, n_samples):
	# choose random instances
	ix = randint(0, dataset.shape[0], n_samples)
	# retrieve selected images
	X = dataset[ix]
	# generate 'real' class labels (1)
	y = ones((n_samples, 1))
	return X, y
 
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
	# generate points in the latent space
	x_input = randn(latent_dim * n_samples)
	# reshape into a batch of inputs for the network
	x_input = x_input.reshape(n_samples, latent_dim)
	return x_input
 
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(g_model, latent_dim, n_samples, covert_msg):
	# generate points in latent space
	l_points = generate_latent_points(latent_dim, n_samples)
	x_input = np.concatenate([covert_msg, l_points], axis=1)
	# predict outputs
	X = g_model.predict(x_input)
	# create 'fake' class labels (0)
	y = zeros((n_samples, 1))
	return X, y

In [ ]:
def generate_covert_samples(n_samples):
    covert = np.random.randint(2,size=n_samples)
    covert_msg = np.array([[c, int(not(c))] for c  in covert], dtype=np.float32)
    covert_output = np.zeros((covert.shape[0], output_dim))
    for i in range(covert.shape[0]):
      if (covert[i]==0):
        covert_output[i][TARGET_INDEX1]=60
      if (covert[i]==1):
        covert_output[i][TARGET_INDEX2]=60
    return covert_msg, covert_output

# a function for plots

In [ ]:
# plot the generated images
def create_plot(examples, n):
	# plot images
	for i in range(n * n):
		# define subplot
		pyplot.subplot(n, n, 1 + i)
		# turn off axis
		pyplot.axis('off')
		# plot raw pixel data
		pyplot.imshow(examples[i, :, :])
	pyplot.show()

# definition of custom loss

In [ ]:
BATCH_SIZE_PER_REPLICA = 16
GLOBAL_BATCH_SIZE = BATCH_SIZE_PER_REPLICA * tpu_strategy.num_replicas_in_sync
with tpu_strategy.scope():
    # Set reduction to `none` so we can do the reduction afterwards and divide by
    # global batch size.
    mae = tf.keras.losses.MeanAbsoluteError(
          reduction=tf.keras.losses.Reduction.NONE)
    def Covert_customLoss(yTrue,yPred):
        per_example_loss = mae(yTrue,yPred*mask)
        return tf.nn.compute_average_loss(per_example_loss, global_batch_size=GLOBAL_BATCH_SIZE)

# Covert model definition

In [ ]:
from tensorflow.keras.optimizers import Adam

def define_covert_model(i_model, g_model):
    i_model.trainable = False
    intermediate_layer_model = tf.keras.Model(inputs=i_model.input,
                                       outputs=i_model.get_layer(index=-2).output)

    c_model = Sequential()
    c_model.add(g_model)
    c_model.add(intermediate_layer_model)
    opt = Adam(learning_rate=0.002)
    c_model.compile(loss=Covert_customLoss,
                    optimizer=opt)
    return c_model

In [ ]:
def train_covert_model(_size):
  c_msg, c_out = generate_covert_samples(_size)
  l_points = generate_latent_points(latent_dim, _size)
  c_l = np.concatenate([c_msg, l_points], axis=1)   
  c_loss = c_model.fit(c_l,c_out,batch_size=32)
  return c_loss

# a function for discriminator evaluation, plot generated images, save generator model

In [ ]:
# evaluate the discriminator, plot generated images, save generator model
def summarize_performance(epoch, g_model, d_model, dataset, latent_dim, n_samples=150):
	covert = np.random.randint(2,size=n_samples)
	covert_msg = np.array([[c, int(not(c))] for c  in covert], dtype=np.float32)
	# prepare real samples
	X_real, y_real = generate_real_samples(dataset, n_samples)
	# evaluate discriminator on real examples
	_, acc_real = d_model.evaluate(X_real, y_real, verbose=0)
	# prepare fake examples
	x_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_samples, covert_msg)
	# evaluate discriminator on fake examples
	_, acc_fake = d_model.evaluate(x_fake, y_fake, verbose=0)
	# summarize discriminator performance
	print('>Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real*100, acc_fake*100))

# some functions for saving generators and showing results

In [ ]:
def save_function(epoch):
	filename = "covert_stanford_dogs_generator_model_%03d.h5" % (epoch+1)
	g_model.save(model_path + filename)

In [ ]:
def show_GAN_images(epoch, n_batch):
  # load model
  filename = "covert_stanford_dogs_generator_model_%03d.h5" % (epoch+1)
  model = load_model(model_path + filename)
  # generate images
  latent_points = generate_latent_points(latent_dim, n_batch)
  c_msg, c_out = generate_covert_samples(n_batch)
  c_l = np.concatenate([c_msg, latent_points], axis=1)
  # generate images
  X = model.predict(c_l)
  # scale from [-1,1] to [0,1]
  X = (X + 1) / 2.0
  # plot the result
  create_plot(X, 10)

In [ ]:
def show_covert_indexes(c_model):
  c_msg, c_out = generate_covert_samples(20)
  l_points = generate_latent_points(latent_dim, 20)
  c_l = np.concatenate([c_msg, l_points], axis=1)
  covert_out = c_model.predict(c_l)
  for k in range(20):
    print(c_msg[k], "   ",
          covert_out[k][TARGET_INDEX1], "   ",
          covert_out[k][TARGET_INDEX2], "   ",
          )

# Indicator function definition

In [ ]:
def indicator(c_msg, real_outputs):
  _size = c_msg.shape[0]
  y_pred=[None]*_size
  y_true=[None]*_size
  for i in range(_size):

    if(all(c_msg[i]==[0,1])):
      y_true[i]=0
    elif(all(c_msg[i]==[1,0])):
      y_true[i]=1
    else:
      y_true[i]=2

    ro1=real_outputs[i][TARGET_INDEX1]
    ro2=real_outputs[i][TARGET_INDEX2]
    if (abs(ro1)>5 and abs(ro2)<0.5):
      y_pred[i]=0
    elif (abs(ro1)<0.5 and abs(ro2)>5):
      y_pred[i]=1
    else:
      y_pred[i]=2
      
  return y_true,y_pred

In [ ]:
from sklearn.metrics import confusion_matrix

def show_indicator(c_model):
  c_msg, c_out = generate_covert_samples(1000)
  l_points = generate_latent_points(latent_dim, 1000)
  c_l = np.concatenate([c_msg, l_points], axis=1)
  covert_out = c_model.predict(c_l)

  y_true, y_pred = indicator(c_msg, covert_out)
  print(confusion_matrix(y_true, y_pred))

# this function includes the training loops

In [ ]:
# train the generator and discriminator
def train(c_model, g_model, d_model, gan_model, dataset, latent_dim, n_epochs=200, n_batch=128):
  bat_per_epo = int(dataset.shape[0] / n_batch)
  half_batch = int(n_batch / 2)
	# manually enumerate epochs
  for i in range(n_epochs):
		# enumerate batches over the training set
    d1=0
    d2=0
    g=0
    for j in range(bat_per_epo):
      c_msg, c_out = generate_covert_samples(n_batch)

			# get randomly selected 'real' samples
      X_real, y_real = generate_real_samples(dataset, half_batch)
      # update discriminator model weights
      d_loss1, _ = d_model.train_on_batch(X_real, y_real)
      # generate 'fake' examples
      X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch, c_msg[:half_batch])
      # update discriminator model weights
      d_loss2, _ = d_model.train_on_batch(X_fake, y_fake)

      # prepare points in latent space as input for the generator
      l_gan = generate_latent_points(latent_dim, n_batch)
      X_gan = np.concatenate([c_msg, l_gan], axis=1)
      # create inverted labels for the fake samples
      y_gan = ones((n_batch, 1))
      # update the generator via the discriminator's error
      g_loss = gan_model.train_on_batch(X_gan, y_gan)

      d1=d1+d_loss1
      d2=d2+d_loss2
      g=g+g_loss

      # summarize loss on this batch
      if(j%40==0):
        print('>%d, %d/%d, d1=%.3f, d2=%.3f, g=%.3f' %(i+1, j+1, bat_per_epo, d_loss1, d_loss2, g_loss))
    
    # train covert model using fit function
    _size=10000
    c_loss = train_covert_model(_size)

    print('avg d1=%.3f, avg d2=%.3f, avg g=%.3f' %(d1/bat_per_epo, d2/bat_per_epo, g/bat_per_epo))
    summarize_performance(i, g_model, d_model, dataset, latent_dim)

    if(i%5==0):
      save_function(i)
      show_GAN_images(i, n_batch)
      show_covert_indexes(c_model)
      show_indicator(c_model)

# now define the models and call train function

In [ ]:
model_path = "/content/drive/MyDrive/covert_models/covert_stanford_dogs_generator/"

with tpu_strategy.scope():
  # size of the latent space
  latent_dim = 10
  d_model = define_discriminator()
  g_model = define_generator(latent_dim)
  gan_model = define_gan(g_model, d_model)
  c_model = define_covert_model(isolated_model, g_model)
  train(c_model, g_model, d_model, gan_model, dataset, latent_dim)